# Extractions

The objective of this notebook is to generate extractions from the verbatims.

In [1]:
import pandas as pd
import json

from tqdm import tqdm
from typing import List, Dict
from pymongo import MongoClient

import boto3

from utils.extractions_utils import generate_extractions, split_text_parts, get_extractions, add_extractions_to_splitted_analysis

In [2]:
STAGE = 'prod'

Loading all key libraries

In [3]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"{STAGE.capitalize()}/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)

OPENAI_API_KEY = _secrets["openai"]["api_key"]
LLM_API_KEY = _secrets["litellm"]["api_key"]


In [4]:
mongo_client = MongoClient(MONGO_CONNECTION_STRING)

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

Define the brand and a short description of the brand.

Here `columbuscafe_test` is a brand for testing purposes.

In [5]:
BRAND = 'columbuscafe_test'

BRAND_DESCR = '''
Feedbacks are from client of columbus cafe.
Columbus Café & Co est une chaîne française de cafés.
'''

In [6]:
from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
])))

from_mongo.shape

(2994, 14)

Displaying a random verbatim to understand the data.

In [7]:
row = from_mongo.sample().iloc[0]
text = row.verbatim['text']
#text = row['commentaires']
print(text)

Un lieu qui été bien meilleure au début... Tout nouveau tout beau ....
Personnel toujours agréable mais ...
Produits médiocre un café qui a la couleur du thé .
Un latté de l'ours qui a peu de goût contrairement à celui de calais ....
Et pour finir un restaurant qui devient vetuste et dont certaines odeurs laisse a désirer .
En résumé dernière visite dans ce colombus.


First we will split our text into different parts by using punctuation.

In [8]:
text_parts = split_text_parts(text)

text_parts

[{'text': 'Un lieu qui été bien meilleure au début...'},
 {'text': ' Tout nouveau tout beau ....\n'},
 {'text': 'Personnel toujours agréable mais ...\n'},
 {'text': 'Produits médiocre un café qui a la couleur du thé .\n'},
 {'text': "Un latté de l'ours qui a peu de goût contrairement à celui de calais ....\n"},
 {'text': 'Et pour finir un restaurant qui devient vetuste et dont certaines odeurs laisse a désirer .\n'},
 {'text': 'En résumé dernière visite dans ce colombus.'}]

Then this is an example of generating extractions from the text parts.

In [9]:
result = get_extractions(
    text=text,
    id=row['id'],
    brand_descr=BRAND_DESCR,
    language='french',
    model='gpt-4o-mini'
)

extractions = result.get('extraction')

splitted_analysis = result.get('splitted_analysis')

In [10]:
extractions

[{'sentiment': 'NEGATIVE',
  'extraction': "Lieu moins bon qu'au début",
  'text': 'Un lieu qui été bien meilleure au début...'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Personnel toujours agréable',
  'text': 'Personnel toujours agréable mais ...\n'},
 {'sentiment': 'NEGATIVE',
  'extraction': 'Produits médiocres',
  'text': 'Produits médiocre un café qui a la couleur du thé .\n'},
 {'sentiment': 'NEGATIVE',
  'extraction': "Latté de l'ours peu de goût",
  'text': "Un latté de l'ours qui a peu de goût contrairement à celui de calais ....\n"},
 {'sentiment': 'NEGATIVE',
  'extraction': 'Restaurant vétuste avec des odeurs désagréables',
  'text': 'Et pour finir un restaurant qui devient vetuste et dont certaines odeurs laisse a désirer .\n'},
 {'sentiment': 'NEGATIVE',
  'extraction': 'Dernière visite décevante',
  'text': 'En résumé dernière visite dans ce colombus.'}]

In  `splitted_analysis` we reassemble the extractions with the corresponding text parts

In [11]:
splitted_analysis

[{'text': 'Un lieu qui été bien meilleure au début...',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': "Lieu moins bon qu'au début"}]},
 {'text': ' Tout nouveau tout beau ....\n'},
 {'text': 'Personnel toujours agréable mais ...\n',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Personnel toujours agréable'}]},
 {'text': 'Produits médiocre un café qui a la couleur du thé .\n',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Produits médiocres'}]},
 {'text': "Un latté de l'ours qui a peu de goût contrairement à celui de calais ....\n",
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': "Latté de l'ours peu de goût"}]},
 {'text': 'Et pour finir un restaurant qui devient vetuste et dont certaines odeurs laisse a désirer .\n',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Restaurant vétuste avec des odeurs désagréables'}]},
 {'text': 'En résumé dernière visite dans ce colombus.',
  'extractions': [{'sentiment': 'NEGA

## Run the extraction pipeline

In [12]:
# this function allows to parallelize the extraction process and to save the results on the mongo database
from utils.extractions_utils import run_extractions_full_parallel

In [ ]:
subdf = from_mongo.sample(20) # test on a subset
print('To extract:', subdf.shape[0])

subdf['text'] = subdf.verbatim.apply(lambda x: x['text'] if isinstance(x, dict) and 'text' in x else '',)
texts_with_ids = subdf[['text', '_id']].to_dict(orient='records')

In [15]:
extractions = run_extractions_full_parallel(
    texts_with_ids,
    BRAND,
    brand_descr=BRAND_DESCR,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=True
)

Processing chunks:   0%|          | 0/1 [00:00<?, ?it/s]

[parse_extraction()] Il semble que vous n'ayez pas fourni de texte à analyser. Veuillez soumettre le texte contenant les retours des clients de Columbus Café & Co, et je pourrai alors extraire les informations pertinentes en suivant vos instructions. unexpected EOF while parsing (<string>, line 0)
 columbuscafe_test/1fd3f4ab1be2aaacc937
[get_extractions()] 'text'


Processing chunks: 100%|██████████| 1/1 [00:07<00:00,  7.26s/it]


In [16]:
extractions

[{'id': 'columbuscafe_test/f6790519b4ff3fd7b410',
  'splitted_analysis': [{'text': 'Je viens régulièrement prendre mon petit moka au colombus et c’est toujours un plaisir d’être reçu par Iris qui est souriante et serviable !',
    'extractions': [{'sentiment': 'POSITIVE',
      'extraction': "Satisfaction de l'accueil par Iris"}]},
   {'text': ' De plus,'},
   {'text': ' son moka est exceptionnel 🤩',
    'extractions': [{'sentiment': 'POSITIVE',
      'extraction': 'Moka exceptionnel'}]}],
  'extraction': [{'sentiment': 'POSITIVE',
    'extraction': "Satisfaction de l'accueil par Iris",
    'text': 'Je viens régulièrement prendre mon petit moka au colombus et c’est toujours un plaisir d’être reçu par Iris qui est souriante et serviable !'},
   {'sentiment': 'POSITIVE',
    'extraction': 'Moka exceptionnel',
    'text': ' son moka est exceptionnel 🤩'}]},
 {'id': 'columbuscafe_test/a965ce53ffc4263dd876',
  'splitted_analysis': [{'text': 'Très propre le personnel est très aimable je vous 

## Checking result in MongoDB

In [36]:
result

{'id': '1c59e5e7bcf3acca00b9',
 'splitted_analysis': [{'text': 'Expérience too good to go plus que positive.',
   'extractions': [{'sentiment': 'POSITIVE',
     'extraction': 'Expérience positive avec too good to go'}]},
  {'text': ' Personnel agréable et serviable.',
   'extractions': [{'sentiment': 'POSITIVE',
     'extraction': 'Personnel agréable et serviable'}]},
  {'text': " J'ai eut de bons conseils pour la conservation des muffins.\n",
   'extractions': [{'sentiment': 'POSITIVE',
     'extraction': 'Bons conseils pour la conservation des muffins'}]},
  {'text': 'Merci.'}],
 'extraction': [{'sentiment': 'POSITIVE',
   'extraction': 'Expérience positive avec too good to go',
   'text': 'Expérience too good to go plus que positive.'},
  {'sentiment': 'POSITIVE',
   'extraction': 'Personnel agréable et serviable',
   'text': ' Personnel agréable et serviable.'},
  {'sentiment': 'POSITIVE',
   'extraction': 'Bons conseils pour la conservation des muffins',
   'text': " J'ai eut de b

In [18]:
# getting the document from the database to check if the extractions are saved
# matching the brand and the id in res

documents = collection.find({
    'brand': BRAND,
    '_id': {'$in': [r['id'] for r in extractions]}
})

documents = pd.DataFrame(documents)

In [19]:
documents

,_id,id,brand,timestamp,verbatim,establishment,review_site,author,rating_out_of_5,language,extractions,splitted_analysis_v2,review_title,splitted_analysis,topics
0,columbuscafe_test/09849c25a5224e258d60,09849c25a5224e258d60,columbuscafe_test,1.682640e+12,"{'text': 'Très bon accueil, très bon service. ...",Carcassonne Pont Rouge,Google,Sophie Chioccioli,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Très...","[{'text': 'Très bon accueil,', 'extractions': ...",NaN,NaN,NaN
1,columbuscafe_test/13892e2103825cf06d02,13892e2103825cf06d02,columbuscafe_test,1.690416e+12,{'text': '1👍'},Aix-en-Provence Masse,Uber Eats,Ambre G,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Appr...","[{'text': '1👍', 'extractions': [{'sentiment': ...",27-07-2023 - 14.8EUR,NaN,NaN
2,columbuscafe_test/1de034c59bebb207d616,1de034c59bebb207d616,columbuscafe_test,1.691885e+12,{'text': 'Personnel toujours agréable. Service...,Toulouse Compans,Google,Justine Pécheu,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Pers...","[{'text': 'Personnel toujours agréable. ', 'ex...",NaN,NaN,NaN
3,columbuscafe_test/1fd3f4ab1be2aaacc937,1fd3f4ab1be2aaacc937,columbuscafe_test,1.683850e+12,{},Collégien Bay 2,Google,nabia klai,3.0,fr,[],[],NaN,NaN,NaN
4,columbuscafe_test/43f07543c941a5d88922,43f07543c941a5d88922,columbuscafe_test,1.681344e+12,{'text': 'Service courtois. Préparation des be...,Vitrimont,Google,Steve Boudrias,4.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Serv...","[{'text': 'Service courtois.', 'extractions': ...",NaN,NaN,NaN
5,columbuscafe_test/5134f5a2d1a4b013659f,5134f5a2d1a4b013659f,columbuscafe_test,1.689293e+12,{'text': 'Nourriture bonne et la personne au c...,Gemo Mondeville,Google,aicha elackal,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Nour...",[{'text': 'Nourriture bonne et la personne au ...,NaN,[{'text': 'Nourriture bonne et la personne au ...,"[{'topic': {'1': 'Personnel', '2': 'Aimabilité..."
6,columbuscafe_test/5b6e404652649c8ae22e,5b6e404652649c8ae22e,columbuscafe_test,1.696810e+12,{'text': '2👍'},Lyon Frères Lumière,Uber Eats,Thomas B,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Appr...","[{'text': '2👍', 'extractions': [{'sentiment': ...",08-10-2023 - 15.7EUR,NaN,NaN
7,columbuscafe_test/699a0ea75c0b3e677a19,699a0ea75c0b3e677a19,columbuscafe_test,1.690502e+12,{'text': 'Super lieux je recommande'},Valenciennes Place d'Armes,Google,francois drapier,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Reco...","[{'text': 'Super lieux je recommande', 'extrac...",NaN,NaN,NaN
8,columbuscafe_test/6d79829dfc75c2e4ed36,6d79829dfc75c2e4ed36,columbuscafe_test,1.689120e+12,"{'text': 'Toujours très bien accueillis, Une é...",Carcassonne Pont Rouge,Google,K B,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Accu...","[{'text': 'Toujours très bien accueillis,', 'e...",NaN,NaN,NaN
9,columbuscafe_test/6e5641974a4d3ce0e1dc,6e5641974a4d3ce0e1dc,columbuscafe_test,1.685232e+12,{'text': 'Les produits sont excellents et l'ac...,Colmar,Google,Muriel FLUCK,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Prod...",[{'text': 'Les produits sont excellents et l'a...,NaN,NaN,NaN


In [20]:
documents.sample().iloc[0].extractions

[{'sentiment': 'POSITIVE',
  'extraction': 'Serveuse souriante',
  'text': 'La serveuse est souriante et rapide.\n'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Serveuse rapide',
  'text': 'La serveuse est souriante et rapide.\n'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Serveuse arrange le client',
  'text': 'Elle n’hésite pas à arranger le client.\n'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Boissons latte et cookies très gourmandes',
  'text': 'Les boissons latte et cookies sont très gourmand.'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Aime le matcha coco glacé',
  'text': ' (Personnellement jadoooooore le matcha coco glacé ❤️)\n'},
 {'sentiment': 'SUGGESTION',
  'extraction': 'Travaux à prévoir dans le lieu',
  'text': ' il y a quelques travaux a prévoir (peinture,'},
 {'sentiment': 'SUGGESTION',
  'extraction': 'Travaux à prévoir dans le lieu',
  'text': ' lumière défaillante).'},
 {'sentiment': 'NEGATIVE',
  'extraction': 'Ruptures de stock',
  'text': ' il y avait d

In [22]:
documents.sample().iloc[0].splitted_analysis_v2

[{'text': "Un peu cher pour des produits qu'ils ne font pas eux même",
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Produits trop chers'},
   {'sentiment': 'NEGATIVE',
    'extraction': 'Produits non fabriqués par Columbus Café'}]}]